In [1]:
import SoftRobots

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.4/SpatialFields.ji for module SpatialFields.


In [17]:
reload("SoftRobots")

In [18]:
function cube()
    k = 10.
    b = 4.0
    l = 1.0
    m = 0.1
    nodes = SoftRobots.PointMass[]
    positions = SoftRobots.Point{3, Float64}[]
    for z = [0; 1]
        for x = [0; 1]
            for y = [0; 1]
                push!(nodes, SoftRobots.PointMass(m))
                push!(positions, SoftRobots.Point{3, Float64}(x, y, z))
            end
        end
    end
    push!(nodes, SoftRobots.PointMass(m))
    push!(positions, SoftRobots.Point{3, Float64}(0.5, 0.5, 0.5))
    num_nodes = length(nodes)
    edges = SoftRobots.DampedSpring[]
    for i = 1:num_nodes
        for j = (i+1):num_nodes
            rest_length = norm(positions[i] - positions[j])
            push!(edges, SoftRobots.DampedSpring(i, j, k, b, rest_length))
        end
    end
    faces = SoftRobots.convex_hull(positions)
    robot = SoftRobots.SoftRobot(nodes, edges, faces)
    velocities = [SoftRobots.Point{3, Float64}(0) for n in robot.nodes]
    state = SoftRobots.SoftRobotState(positions, velocities)
    robot, state
end

cube (generic function with 1 method)

In [19]:
robot, state = cube()

(SoftRobots.SoftRobot{Int64,0}([SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1),SoftRobots.PointMass(0.1)],[SoftRobots.DampedSpring(1,2,10.0,4.0,1.0),SoftRobots.DampedSpring(1,3,10.0,4.0,1.0),SoftRobots.DampedSpring(1,4,10.0,4.0,1.4142135623730951),SoftRobots.DampedSpring(1,5,10.0,4.0,1.0),SoftRobots.DampedSpring(1,6,10.0,4.0,1.4142135623730951),SoftRobots.DampedSpring(1,7,10.0,4.0,1.4142135623730951),SoftRobots.DampedSpring(1,8,10.0,4.0,1.7320508075688772),SoftRobots.DampedSpring(1,9,10.0,4.0,0.8660254037844386),SoftRobots.DampedSpring(2,3,10.0,4.0,1.4142135623730951),SoftRobots.DampedSpring(2,4,10.0,4.0,1.0)  …  SoftRobots.DampedSpring(5,6,10.0,4.0,1.0),SoftRobots.DampedSpring(5,7,10.0,4.0,1.0),SoftRobots.DampedSpring(5,8,10.0,4.0,1.4142135623730951),SoftRobots.DampedSpring(5,9,10.0,4.0,0.8660254037844386),SoftRobots.DampedSpr

In [20]:
SoftRobots.draw(robot, state)

In [43]:
function barrier_mesh(surface::InterpolatingSurface)
    center = mean(surface.points)
    widths = 2 * (maximum(surface.points) - minimum(surface.points))
    lb = center - 0.5*widths
    @show lb
    @show widths
    bounds = HyperRectangle(Vec(lb), Vec(widths))
    @show bounds
    barrier_mesh(bounds, surface)
end


barrier_mesh (generic function with 3 methods)

In [44]:
vertices(barrier_mesh(surface))

lb = FixedSizeArrays.Point{3,Float64}((-0.5,-1.0555555555555556,-0.5))

4528-element Array{FixedSizeArrays.Point{3,Float64},1}:
 Point(1.2548191273210272,1.8992635717654716,1.0)                    
 Point(0.0,1.931425270558764,1.1869808261143195)                     
 Point(0.30000000000000004,-0.03730114562212572,-0.09999999999999998)
 Point(0.6000000000000001,1.9648251387791327,-0.27961930566531173)   
 Point(0.7875615235149274,1.9320059679593717,1.3)                    
 Point(1.067765955046174,2.012210399490619,0.8677659550461738)       
 Point(0.09999999999999998,2.095527695532481,0.5510832510880361)     
 Point(0.19999999999999996,1.9960079856770652,1.1515635412326208)    
 Point(1.5,0.5058518373626364,-0.3)                                  
 Point(1.4999810246283858,1.2444444444444442,1.5)                    
 Point(0.30000000000000004,-0.14784841263812898,0.7)                 
 Point(0.8035632335231631,-0.05555555555555558,1.0035632335231632)   
 Point(0.6000000000000001,1.9170917060801067,1.3726472616356624)     
 ⋮                                


widths = FixedSizeArrays.Point{3,Float64}((2.0,4.0,2.0))
bounds = GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((-0.5,-1.0555555555555556,-0.5)),FixedSizeArrays.Vec{3,Float64}((2.0,4.0,2.0)))


In [45]:
function draw(surface)
    lcm = SoftRobots.LCM()
    msg = SoftRobots.lcmdrake[:lcmt_viewer_load_robot]()
    push!(msg["link"], SoftRobots.link_data(barrier_mesh(surface), "barrier", 1))
    msg[:num_links] = 1
    SoftRobots.publish(lcm, "DRAKE_VIEWER_LOAD_ROBOT", msg)
end

draw (generic function with 1 method)

In [46]:
draw(surface)

lb = FixedSizeArrays.Point{3,Float64}((-0.5,-1.0555555555555556,-0.5))
widths = FixedSizeArrays.Point{3,Float64}((2.0,4.0,2.0))
bounds = GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((-0.5,-1.0555555555555556,-0.5)),FixedSizeArrays.Vec{3,Float64}((2.0,4.0,2.0)))


In [48]:
evaluate(surface, [1; 2.; 0])

-8.881784197001252e-16